In [1]:
import numpy as np
from tqdm import tqdm
from pycocotools.coco import COCO
from get_obb_gbb_from_coco import process_annotations

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
ann_file = '/datasets/dataset/coco2017/annotations/instances_train2017.json'
coco = COCO(ann_file)
count = 0

loading annotations into memory...
Done (t=13.09s)
creating index...
index created!


In [3]:
voc_cats = [
    'airplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'dining table',
    'dog', 'horse', 'motorcycle', 'person', 'potted plant', 'sheep', 'sofa', 'train', 'tv'
]
catIds = coco.getCatIds(catNms=voc_cats)

name_id = dict(zip(catIds, voc_cats))

In [5]:
with open(ann_file.replace('.json', '_gbb.csv'), 'w') as file:
    
    file.write('file_name,label,x_mean,y_mean,cov00,cov11,cov01\n')
    
    for index in tqdm(coco.imgs):
        count += 1
        img_metadata = coco.loadImgs(index)[0]
        cocotarget   = coco.loadAnns(coco.getAnnIds(imgIds=index))

        obb, gbb, labels = process_annotations(cocotarget, img_metadata['height'], img_metadata['width'], showResults = False)
        
        if not all([l in catIds for l in labels]):
            continue
        
        for g,l in zip(gbb,labels):
            file.write(f"{img_metadata['file_name']},{name_id[l]},{g[0]},{g[1]},{g[2]},{g[3]},{g[4]}\n")

100%|██████████| 118287/118287 [36:26<00:00, 54.09it/s] 


In [6]:
ann_file = '/datasets/dataset/coco2017/annotations/instances_val2017.json'
coco = COCO(ann_file)
count = 0

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!


In [7]:
with open(ann_file.replace('.json', '_gbb.csv'), 'w') as file:
    
    file.write('file_name,label,x_mean,y_mean,cov00,cov11,cov01\n')
    
    for index in tqdm(coco.imgs):
        count += 1
        img_metadata = coco.loadImgs(index)[0]
        cocotarget   = coco.loadAnns(coco.getAnnIds(imgIds=index))

        obb, gbb, labels = process_annotations(cocotarget, img_metadata['height'], img_metadata['width'], showResults = False)
        
        if not all([l in catIds for l in labels]):
            continue
        
        for g,l in zip(gbb,labels):
            file.write(f"{img_metadata['file_name']},{name_id[l]},{g[0]},{g[1]},{g[2]},{g[3]},{g[4]}\n")

100%|██████████| 5000/5000 [01:34<00:00, 53.12it/s]


In [9]:
import pandas as pd

In [13]:
df = pd.read_csv('/datasets/dataset/coco2017/annotations/instances_train2017_gbb.csv')
df

,file_name,label,x_mean,y_mean,cov00,cov11,cov01
0,000000391895.jpg,boat,416.0,240.0,2725.663523,425.374931,-71.894019
1,000000391895.jpg,airplane,482.0,193.0,142.108975,50.597579,-21.737062
2,000000391895.jpg,bicycle,498.0,201.0,84.194703,44.438205,-28.068038
3,000000204805.jpg,cat,204.0,203.0,1804.455975,16008.309572,1950.677239
4,000000204805.jpg,airplane,417.0,188.0,63.865470,45.957851,2.849276
...,...,...,...,...,...,...,...
99013,000000019181.jpg,sheep,48.0,462.0,243.401456,155.631169,-140.680504
99014,000000226055.jpg,airplane,498.0,272.0,6304.851984,6151.795556,-1190.665289
99015,000000180682.jpg,person,414.0,368.0,919.671157,126.878532,-36.145204
99016,000000180682.jpg,person,454.0,343.0,2210.765503,169.887779,11.958766


In [4]:
voc_ids = dict(zip(voc_cats, range(1,len(voc_cats)+1)))

with open('/datasets/dataset/coco2017/annotations/classes_gbb.csv', 'w') as file:
    
    file.write('name,id\n')
    
    for name,id in voc_ids.items():
        file.write(f'{name},{id}\n')